In [1]:
import numpy as np
import torch
from torchvision.datasets import mnist
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

In [2]:
import time
batch_size=256
test_dataset = mnist.MNIST(root='../../test', train=False, transform=ToTensor())
test_loader = DataLoader(test_dataset, batch_size=batch_size)
def calc():
    all_correct_num = 0
    all_sample_num = 0
    model.eval()
    t_ls=[]
    fps_ls=[]
    for idx, (test_x, test_label) in enumerate(test_loader):
        start = time.time()
        predict_y = model(test_x.float()).detach()
        end = time.time()
        total_time = (end - start)
        fps=1/(total_time)
        fps_ls.append(fps)
        t_ls.append(total_time)
        predict_y = np.argmax(predict_y, axis=-1)
        current_correct_num = predict_y == test_label
        all_correct_num += np.sum(current_correct_num.numpy(), axis=-1)
        all_sample_num += current_correct_num.shape[0]
    t=sum(t_ls)
    fps=sum(fps_ls)/len(fps_ls)
    avg_time=t/len(t_ls)
    acc = all_correct_num / all_sample_num
    return t,avg_time,fps,acc

In [3]:
model=torch.load("m2_0.99.pkl")
calc()

(0.34807848930358887, 0.008701962232589722, 139.18321491972227, 0.9914)

In [4]:
from torchsummary import summary
summary(model,(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 7, 22, 22]             350
              ReLU-2            [-1, 7, 22, 22]               0
         MaxPool2d-3            [-1, 7, 11, 11]               0
            Conv2d-4             [-1, 18, 9, 9]           1,152
              ReLU-5             [-1, 18, 9, 9]               0
         MaxPool2d-6             [-1, 18, 4, 4]               0
            Linear-7                  [-1, 120]          34,680
              ReLU-8                  [-1, 120]               0
            Linear-9                   [-1, 84]          10,164
             ReLU-10                   [-1, 84]               0
           Linear-11                   [-1, 10]             850
             ReLU-12                   [-1, 10]               0
Total params: 47,196
Trainable params: 47,196
Non-trainable params: 0
---------------------------------

In [9]:
s=0
a=0
for i in range(100):
    temp=calc()
    s+=temp[1]
    a+=temp[2]
print(s/100,a/100)

9471.409759577678 0.9913999999999991


In [11]:
s=0
a=0
s1=0
for i in range(100):
    temp=calc()
    s1+=temp[0]
    s+=temp[1]
    a+=temp[2]
print(s1/100,s/100,a/100)

1.063879210948944 9415.690773965602 0.9913999999999991


In [12]:
1/(1.063879210948944/40)

37.598253249371574